In [19]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report


In [4]:
data = pd.read_csv('ecommerceDataset.csv')

In [5]:
data.sample(10)

,Household,"Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal bliss.so bring home this elegant print that is lushed with rich colors that makes it nothing but sheer elegance to be to your friends and family.it would be treasured forever by whoever your lucky recipient is. Liven up your place with these intriguing paintings that are high definition hd graphic digital prints for home, office or any room."
9583,Household,Bhagya Cast Iron Cookware Skillet Frying Pan -...
12316,Household,Crazy Sutra 100Ml Premium Glass Bottle With Co...
18202,Household,Homdum® Heavy Duty Cast Iron Base Drill Stand ...
35172,Clothing & Accessories,DFY Men's Track Jacket DFY sweat - garmets spe...
34968,Clothing & Accessories,RG Designers Men's Orange Cotton Kurta Pajama ...
8242,Household,Styleys 6 Pairs Shoes Storage Box Shoe Organiz...
27340,Books,Baby Touch: Playbook About the Author Lemon Ri...
16420,Household,Wishpool Broom and Dustpan Treelen with Dust P...
19,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
27067,Books,Biotechnology: Expanding Horizons Biotechnolog...


In [9]:
data.columns = ['Category','Text']

In [11]:
data.sample(10)

,Category,Text
37039,Clothing & Accessories,Xs and Os Women Designer Lace Thong Lace panty...
41082,Electronics,Creative Sound Blaster Omni 5.1 (SB-OMN-51) Cr...
31314,Clothing & Accessories,iSweven Fashionable Woolen Beanie Cap for Wome...
37531,Clothing & Accessories,AV2 Women's Cotton Solid Feeding/Nursing/Mater...
18353,Household,Venus VJP-475/7 Iron Block Plane (Grey) Block ...
8115,Household,Kids Mandi Polyester Laundry Basket with Zippe...
6303,Household,Lifestyle-You PVC Static Decorative Frosted No...
44585,Electronics,iBall Soundstick BT14 Bluetooth Speakers with ...
5162,Household,Decals Design ' Bird House on a Branch' Wall S...
40699,Electronics,SanDisk 64GB Class 10 microSDXC Memory Card wi...


In [13]:
data['Category'].value_counts()

Category
Household                 19312
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [15]:
X = data['Text']
y = data['Category']

In [17]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [21]:
X_train = X_train.fillna('') 
X_test = X_test.fillna('') 

In [23]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

In [28]:
model = Sequential()
model.add(Dense(128, input_dim=X_train_tfidf.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

C:\Users\213025\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [32]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [34]:
model.fit(X_train_tfidf, y_train, epochs=5, batch_size=16, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/5
2017/2017 ━━━━━━━━━━━━━━━━━━━━ 144s 70ms/step - accuracy: 0.8734 - loss: 0.3637 - val_accuracy: 0.9755 - val_loss: 0.1013
Epoch 2/5
2017/2017 ━━━━━━━━━━━━━━━━━━━━ 134s 66ms/step - accuracy: 0.9916 - loss: 0.0311 - val_accuracy: 0.9755 - val_loss: 0.1040
Epoch 3/5
2017/2017 ━━━━━━━━━━━━━━━━━━━━ 134s 66ms/step - accuracy: 0.9965 - loss: 0.0129 - val_accuracy: 0.9773 - val_loss: 0.1228
Epoch 4/5
2017/2017 ━━━━━━━━━━━━━━━━━━━━ 133s 66ms/step - accuracy: 0.9973 - loss: 0.0089 - val_accuracy: 0.9766 - val_loss: 0.1270


In [36]:
loss, accuracy = model.evaluate(X_test_tfidf, y_test)
print(f"Тест точност: {accuracy:.4f}")

316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9766 - loss: 0.0837
Тест точност: 0.9768


In [38]:
y_pred = model.predict(X_test_tfidf)
y_pred_classes = y_pred.argmax(axis=1)


316/316 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [40]:
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))


                        precision    recall  f1-score   support

                 Books       0.98      0.97      0.97      2379
Clothing & Accessories       0.98      0.98      0.98      1750
           Electronics       0.97      0.97      0.97      2082
             Household       0.98      0.98      0.98      3874

              accuracy                           0.98     10085
             macro avg       0.98      0.98      0.98     10085
          weighted avg       0.98      0.98      0.98     10085

